# TerraClimate

In [ ]:
# Import relevant libraries
import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()  # Only needed for the first time, follow the prompt to authenticate
ee.Initialize()

In [ ]:
# define region
region = []

# Define the date range
start_date = '2014-06-01'
end_date = '2023-09-01'

# Load the TerraClimate dataset from Earth Engine and filter according to date and region (point coords)
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
    .filterBounds(region) \
    .filterDate(start_date, end_date)

# Select the variable of interest (monthly evapotranspiration)
evapotranspiration = terraclimate.select('aet')

In [ ]:
# Define a function to acquire monthly evapotranspiration
def calculate_monthly_et(image):
    # Reduce the region to get the monthly ET within the point/region
    monthly_et = image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=4640
    ).get('aet')

    # Apply the scaling factor
    scaled_et = ee.Number(monthly_et).multiply(0.1)

    # Get the date of the image
    date = image.date().format("YYYY-MM")

    # Return the date and the monthly ET as properties of a feature
    return ee.Feature(None, {
        'date': date,
        'evapotranspiration': scaled_et
    })

In [ ]:
# Convert the result to a list to export it as a DataFrame
et_list = evapotranspiration_data.aggregate_array('evapotranspiration').getInfo()
dates_list = evapotranspiration_data.aggregate_array('date').getInfo()

In [ ]:
# Combine dates and ET data into a DataFrame
et_df = pd.DataFrame({
    'date': dates_list,
    'evapotranspiration_TC': et_list
})

# Export the DataFrame to CSV
et_df.to_csv('monthly_terraclimate_data.csv', index=False)
print("CSV file has been created.")